In [10]:
from tqdm import tqdm
import asyncio
from pymongo import MongoClient
from pymongo import UpdateOne
from src.MongoDB_Controller import MongoDBController
import pandas as pd
import numpy as np

In [3]:
batch_size = 100000
client = MongoClient("mongodb://localhost:27017/")
mongo = MongoDBController(client, batch_size)

In [4]:
cursor = mongo['riot_match_modv2'].find({}, {'_id': 1, 'summonerId': 1, 'teamId': 1, 'individualPosition': 1, 'matchId': 1})

df = pd.DataFrame(list(cursor))
df

,_id,summonerId,teamId,individualPosition,matchId
0,6600fa86106042e6ad6c4375,75508,2,2,63886
1,6600fa86106042e6ad6c4376,68758,1,3,63886
2,6600fa86106042e6ad6c4377,259395,2,4,96632
3,6600fa86106042e6ad6c4378,53070,1,1,59625
4,6600fa86106042e6ad6c4379,181933,2,2,6795
...,...,...,...,...,...
1319605,6600fb00106042e6ad80662a,196469,2,2,89556
1319606,6600fb00106042e6ad80662b,245353,2,2,21673
1319607,6600fb00106042e6ad80662c,54671,2,3,21673
1319608,6600fb00106042e6ad80662d,111356,2,4,21673


In [5]:
### 중복 행 제거
df = df.drop_duplicates(subset=['summonerId', 'matchId'])
df

,_id,summonerId,teamId,individualPosition,matchId
0,6600fa86106042e6ad6c4375,75508,2,2,63886
1,6600fa86106042e6ad6c4376,68758,1,3,63886
2,6600fa86106042e6ad6c4377,259395,2,4,96632
3,6600fa86106042e6ad6c4378,53070,1,1,59625
4,6600fa86106042e6ad6c4379,181933,2,2,6795
...,...,...,...,...,...
1319605,6600fb00106042e6ad80662a,196469,2,2,89556
1319606,6600fb00106042e6ad80662b,245353,2,2,21673
1319607,6600fb00106042e6ad80662c,54671,2,3,21673
1319608,6600fb00106042e6ad80662d,111356,2,4,21673


In [6]:
### 게임의 참여자가 10명인 게임만 추출
temp = df.groupby('matchId').size()
use_index = temp[temp == 10].index
df = df[df.set_index(['matchId']).index.isin(use_index)]

df.reset_index(drop=True, inplace=True)
df

,_id,summonerId,teamId,individualPosition,matchId
0,6600fa86106042e6ad6c4375,75508,2,2,63886
1,6600fa86106042e6ad6c4376,68758,1,3,63886
2,6600fa86106042e6ad6c4377,259395,2,4,96632
3,6600fa86106042e6ad6c4378,53070,1,1,59625
4,6600fa86106042e6ad6c4379,181933,2,2,6795
...,...,...,...,...,...
1287715,6600fb00106042e6ad80662a,196469,2,2,89556
1287716,6600fb00106042e6ad80662b,245353,2,2,21673
1287717,6600fb00106042e6ad80662c,54671,2,3,21673
1287718,6600fb00106042e6ad80662d,111356,2,4,21673


In [7]:
df = df.sort_values(by=['matchId', 'teamId', 'individualPosition'])
df.reset_index(drop=True, inplace=True)

In [8]:
df['position'] = list(range(10)) * (len(df)//10)
df

,_id,summonerId,teamId,individualPosition,matchId,position
0,6600fa8f106042e6ad6e20ba,259351,1,1,1,0
1,6600faad106042e6ad732771,57897,1,2,1,1
2,6600fab7106042e6ad74d7bc,165485,1,3,1,2
3,6600faee106042e6ad7d3c7b,8401,1,4,1,3
4,6600faa1106042e6ad7178e6,174175,1,5,1,4
...,...,...,...,...,...,...
1287715,6600fad3106042e6ad79e1ee,298056,2,1,129121,5
1287716,6600fac0106042e6ad767b8e,134378,2,2,129121,6
1287717,6600faf8106042e6ad7eddfc,257326,2,3,129121,7
1287718,6600faa1106042e6ad717457,26070,2,4,129121,8


In [11]:
# mongo['riot_match_modv2_SAS'].drop()

ids = []
for i in tqdm(df['_id']):
    ids.append(i)

    if len(ids) == batch_size:
        datas = mongo['riot_match_modv2'].find({'_id': {'$in': ids}}, {'_id': 0})
        await mongo.save_to_mongo('riot_match_modv2_SAS', datas)
        ids = []
        
datas = mongo['riot_match_modv2'].find({'_id': {'$in': ids}}, {'_id': 0})
await mongo.save_to_mongo('riot_match_modv2_SAS', datas)

100%|██████████| 1287720/1287720 [01:43<00:00, 12424.84it/s]


In [13]:
bulk_operations = []
for index, row in tqdm(df[['_id', 'position']].iterrows()):
    bulk_operations += [UpdateOne({'_id': row["_id"]}, {'$set': {'position': row["position"]}})]

    if len(bulk_operations) >= batch_size:
        mongo['riot_match_modv2_SAS'].bulk_write(bulk_operations)
        bulk_operations = []

1287720it [01:46, 12140.44it/s]
